# Audio Data Block

In [28]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from audio import *

In [30]:
#Export

#External dependencies
import mimetypes
from fastai.vision import *
import torchaudio
from torchaudio import transforms

#for jupyter Display
from IPython.display import Audio

def source(f):
    print(inspect.getsource(f))

## Sample data for test

In [31]:
# Standard path notation for fast.ai
# The files willbe saved on $HOME/.fastai/data/timit/
path = getFastAiWorkingDirectory('timit')

Working directory: /home/jupyter/.fastai/data/timit


## Data Block classes

In [32]:
DataBunch

fastai.basic_data.DataBunch

### AudioItem
This is the base class of fast.ai Audio tht holds a reference to and AudioData object.

In [33]:
source(AudioItem)

In [34]:
def test_AudioItem_create_from_data():
    signal,samplerate = torchaudio.load(str(path/'TRAIN/DR1/MDPK0/SA1.WAV'))
    a = AudioItem(AudioData(signal,samplerate))
    assert 1 == len(a.data.sig.shape), 'Single dimension data'
    assert a.data.sig.shape[0] > 100, 'Has data'
    assert 16000 == a.data.sr
    display(a)

test_AudioItem_create_from_data()

AudioItem REPRESENTATION

In [35]:
def test_AudioItem_create_from_audio_file():
    a = AudioItem(AudioData.load(str(path/'TRAIN/DR1/MDPK0/SA1.WAV')))
    assert 1 == len(a.data.sig.shape), 'Single dimension data'
    assert a.data.sig.shape[0] > 100, 'Has data'
    assert 16000 == a.data.sr
    display(a)
    
test_AudioItem_create_from_audio_file()

AudioItem REPRESENTATION

## AudioDataBunch

In [36]:
source(AudioDataBunch)

### AudioList
This class is responsible to contain a list of AudioItem.

In [37]:
source(AudioList)

In [38]:
def test_AudioList_from_df_file_names():
    import glob
    #Create Data Frame
    df = pd.DataFrame(glob.glob(str(path/'**/*.WAV'), recursive=True)[:10])
    df.columns = ['FileName']
    display(df.head())

    #Crete AudioList
    ils = AudioList.from_df(df, path, cols=['FileName'])
    
    #Test a item
    i=5
    print(f'FileName: {df.FileName[i]}')
    a = ils.get(i)
    print(a.data.sig.shape, a.data.sr)
    display(a)

test_AudioList_from_df_file_names()

FileName
0  /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS...
1  /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS...
2  /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS...
3  /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS...
4  /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS...

FileName: /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS0/SX43.WAV
torch.Size([39220]) 16000


AudioItem REPRESENTATION

In [39]:
def test_AudioList_from_folder():
    p = path/'TRAIN'

    #Create AudioList
    ils = AudioList.from_folder(p)
    
    #Test an item
    i=4
    a = ils.get(i)
    print(a.data.sig.shape, a.data.sr)
    display(a)

test_AudioList_from_folder()

torch.Size([37991]) 16000


AudioItem REPRESENTATION

In [40]:
def test_AudioList_from_df_data_and_sr():
    import glob
    #Create Data Frame
    df = pd.DataFrame(glob.glob(str(path/'**/*.WAV'), recursive=True)[:10])
    df.columns = ['FileName']
    df['SampleAndSr']=df['FileName'].apply(lambda n: torchaudio.load(n))
    #df = df['tmp'].drop()
    
    display(df.head())

    #Create AudioList
    ils = AudioList.from_df(df, path, cols=['SampleAndSr'])
    
    #Test a item
    i=4
    print(f'FileName: {df.FileName[i]}')
    a = ils.get(i)
    print(a.data.sig.shape, a.data.sr)
    display(a)

test_AudioList_from_df_data_and_sr()

FileName  \
0  /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS...   
1  /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS...   
2  /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS...   
3  /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS...   
4  /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS...   

                                         SampleAndSr  
0  ([[tensor(0.0002), tensor(-0.0002), tensor(-3....  
1  ([[tensor(6.1035e-05), tensor(0.0002), tensor(...  
2  ([[tensor(9.1553e-05), tensor(0.0002), tensor(...  
3  ([[tensor(9.1553e-05), tensor(6.1035e-05), ten...  
4  ([[tensor(3.0518e-05), tensor(0.0002), tensor(...

FileName: /home/jupyter/.fastai/data/timit/TEST/DR1/FAKS0/SX223.WAV
torch.Size([49562]) 16000


AudioItem REPRESENTATION

### Sample AudioDataBunch Usage

In [41]:
import glob

def process_phn_file(p_file, sig, sr, delimiter=' '):
    df = pd.read_csv(p_file, delimiter=delimiter, header=None)
    df.columns = ['Start', 'End', 'Phn']
    df['SampleAndSr'] = df.apply(lambda x : (sig[-1][x['Start']: x['End']], sr), axis=1)
    return df

def create_phn_df(path, count=100):
    phns = []
    final = pd.DataFrame()

    for phn_file in glob.glob(str(path/'**/*.PHN'), recursive=True)[:count]:
        sig,sr = torchaudio.load(str(phn_file.replace('PHN', 'WAV')))
        df = process_phn_file(phn_file, sig, sr, delimiter=' ')
        df['Source'] = phn_file
        final = final.append(df, ignore_index=True)
    return df

df = create_phn_df(path/'TRAIN')
df.head()

Start   End Phn                                        SampleAndSr  \
0      0  2040  h#  ([tensor(-0.0004), tensor(-3.0518e-05), tensor...   
1   2040  2631   w  ([tensor(3.0518e-05), tensor(0.0001), tensor(0...   
2   2631  3258  ih  ([tensor(0.0015), tensor(-0.0002), tensor(-0.0...   
3   3258  3509  nx  ([tensor(0.0076), tensor(0.0072), tensor(0.005...   
4   3509  6015  ao  ([tensor(0.0397), tensor(0.0306), tensor(0.022...   

                                              Source  
0  /home/jupyter/.fastai/data/timit/TRAIN/DR1/MRW...  
1  /home/jupyter/.fastai/data/timit/TRAIN/DR1/MRW...  
2  /home/jupyter/.fastai/data/timit/TRAIN/DR1/MRW...  
3  /home/jupyter/.fastai/data/timit/TRAIN/DR1/MRW...  
4  /home/jupyter/.fastai/data/timit/TRAIN/DR1/MRW...

In [42]:
step0 = AudioList.from_df(df, path, cols=['SampleAndSr']); print(type(step0))

<class '__main__.AudioList'>


In [43]:
step0

AudioList (21 items)
REPRESENTATION,REPRESENTATION,REPRESENTATION,REPRESENTATION,REPRESENTATION
Path: /home/jupyter/.fastai/data/timit

In [44]:
step1 = step0.split_by_rand_pct(0.1, seed=1);
print(f'Result type: {type(step1)}, Type of Items: {type(step1.lists[0])}')

Result type: <class 'fastai.data_block.ItemLists'>, Type of Items: <class '__main__.AudioList'>


In [45]:
step2 = step1.label_from_df('Phn');
print(f'Result type: {type(step2)}')

Result type: <class 'fastai.data_block.LabelLists'>


/opt/anaconda3/envs/fastai-audio/lib/python3.7/site-packages/fastai/data_block.py:522: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
ey
  if getattr(ds, 'warn', False): warn(ds.warn)


In [46]:
source(get_audio_transforms)

In [47]:
batch_size = 8

In [48]:
tfms = get_audio_transforms()
step3 = step2.transform(tfms);
print(type(step3))

<class 'fastai.data_block.LabelLists'>


In [49]:
step4 = step3.databunch(bs=batch_size);
print(type(step4))

<class '__main__.AudioDataBunch'>


In [50]:
step4 = step3.databunch(bs=batch_size);
step4

AudioDataBunch;

Train: LabelList (19 items)
x: AudioList
REPRESENTATION,REPRESENTATION,REPRESENTATION,REPRESENTATION,REPRESENTATION
y: CategoryList
h#,w,ih,nx,ao
Path: /home/jupyter/.fastai/data/timit;

Valid: LabelList (1 items)
x: AudioList
REPRESENTATION
y: CategoryList
z
Path: /home/jupyter/.fastai/data/timit;

Test: None

In [51]:
max_len = 1000
datablock = step4
batch = datablock.one_batch()
print(len(batch[0]), batch[0].shape)
assert batch_size == len(batch[0])
assert max_len == batch[0].shape[1]

8 torch.Size([8, 1000])


In [52]:
# datablock.show_batch()